# Exercise 2 - Get Started with Grounding

In this exercise, we'll start exploring the grounding on SAP AI Core. Grounding allows Large Language Models (LLMs) to reference specific knowledge sources when generating responses, which can help improve accuracy and relevance. 

We'll start with the basics by grounding an LLM using SAP.help as our knowledge source. 

This exercise will cover:

* Basic grounding concepts in SAP AI Core
* How to set up grounding with a sap.help.com
* Comparing responses with and without grounding


## Check your connection to AI Core 
👀  In the ```TECHE2025-167/excercise/init_env.py``` the values from ```TECHED2025-AI167/.aicore-config.json``` are assigned to environmental variables. That way the **SAP Cloud SDK for AI(Python)** will connect to AI Core. 

## Add orchestration deployment URL to variables.py 
👀  Before we can started, one final step is missing. You need to change the value assigened to the **AICORE_ORCHESTRATION_DEPLOYMENT_URL** in the ```variables.py``` to your own orchestration deployment URL.
Therefore run the blow code and copy the URL and add into the variables.py add the respective place.
<p>
<img src="images/deployment_url.png" alt="Visual Studio Code" width="600"/>
</p>

![Deployment URL screenshot](images/deployment_url.png)


In [ ]:
import init_env
import helpers


init_env.set_environment_variables()

url = helpers.extract_deployment_url()
print(url)

➡️ Restart the Juypyter Kernel

<p>
<img src="images/restart_kernel.png" alt="Visual Studio Code" width="800"/>
</p>

In [ ]:
import init_env
import variables
# Run this to test whether all environment variables are set correctly 
init_env.set_environment_variables()

assert variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL!='', """You should change the value assigned to the `AICORE_ORCHESTRATION_DEPLOYMENT_URL` in the `variables.py` file to your own resource group first!"""
print(f"Deployment URL is set to: {variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL}")

> 
> ✨ **More information: Orchestration Service**  
> 
>The orchestration service of Generative AI Hub lets you use all the available models with the same codebase. You only deploy the orchestration service and then you can access all available models simply by changing the model name parameter.   
>You can also use grounding, prompt templating, data masking and content filtering capabilities.  
>In the following we are mainly using it for grounding

## Let's start with a simple prompt

To understand the general idea about grounding a model, let us first start with the simple prompt without grounding it. 

### Import the packages we want to use 

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

### Assign the model we want to use 

In [ ]:
llm = LLM(
    name="gemini-2.5-flash",
    parameters={
        'temperature': 0.0,
    }
)

### Create a prompt Template
The parameter user_query in the code snippet below is going to hold the user query that you will add later on. 

In [ ]:
template = Template(
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                """),
            ]
        )

### Create an orchestration configuration
Next you need to create the orchestration configuration by including the LLM we referencesd and the prompt template we just created.

In [ ]:
config = OrchestrationConfig(
    template=template,
    llm=llm,
)

### Execute the query
This configuration we now add to the OrchestrationService and then we run to retrieve the answer. 

In [ ]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config,
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            #TODO Here you can change the user prompt into whatever you want to ask the model
            value="What is Joule?"
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

### Looks correct, but not for our context

The model’s answer to “What is Joule?” is *technically* correct because, without any extra context, it defaults to the statistically most common public meaning (the physics unit of energy). Our expectation at SAP TechEd was an SAP‑specific interpretation (e.g., an SAP capability or product named “Joule”), but the prompt contained no SAP signals, product names, or retrieved SAP documents to steer the model. With no domain clues, the generic global prior wins.  

Grounding can solve this: attach a grounding/data-retrieval step (SAP Help, internal docs, knowledge base) and inject them into the prompt (or pipeline) before generation. 
In short: no context → generic answer; grounded context → domain‑specific answer.

💪  Let's do this. 

## Ground your prompt with SAP Help

To add context to the retrieval step we need to some change to our original steps. 

Starting at the template by adding to the UserMessage ```Context:{{ ?grounding_response }}```. Whereas **grounding_response** is the context retrieved from the context information, in this case sap.help.com.


In [ ]:

template = Template(
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

### Import the packages we want to use 

In [ ]:

from gen_ai_hub.orchestration.models.document_grounding import DocumentGroundingFilter
from gen_ai_hub.orchestration.models.document_grounding import GroundingModule
from gen_ai_hub.orchestration.models.document_grounding import GroundingType
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding

### Define data repository 
We need to configure the Grounding Module, where we first define the data repository that we want to use via the **filter** parameter. 
  
➡️ Add **"help.sap.com"** as the **data_repository_type** and run the statement. 

In [ ]:

filters = [
            DocumentGroundingFilter(id="SAPHelp", data_repository_type=<data_repository_type>) # replace <data_repository_type> by "help.sap.com"
        ]


### Create Grounding Configuration
Next we create the grounding configuration by using **GroundingModule** for managing and applying grounding configurations:  
- **type**: "document_grounding_service"
- **config**:  Configuration dictionary including parameter defined in the template and filter that includes the data repository type

In [ ]:

grounding_config = GroundingModule(
            type=GroundingType.DOCUMENT_GROUNDING_SERVICE.value,
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

### Create Orchestration Configuration 

Grounding configuration ```grounding_config``` is now an addidional parameter that we add to the Orchestration configuration. 

In [ ]:

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

### Execute the  Query
Configuration will be added again to the OrchestrationService and then we run to retrieve the answer.

In [ ]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue( 
            name="user_query",
            value="What is Joule?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

### The right context
Grounding succeeded: with help.sap.com context injected, the model produced the correct SAP‑specific answer.

## Summary 
You learned the basic grounding concepts in SAP AI Core and how you to use it improve the retrieval.

In the next execercise you will learn on how to injest custom documents. 


Continue to - [Excercise 3: Ground your LLM with custom documents](ex3-1-grounding-vector-api.md)
